Торги акциями происходят в несколько сессий. В этой связи важно скачать котировки акций с учетом всех торгов.

Можно подключиться через ссылку `iss_url`, которая предоставляет данные по итогам всех торгов. В параметрах `start_date` и `end_date` указываем необходимые даты для скачивания (включает как дату начала, так и конечную дату). В `tickers` прописываем тикеры акций с сайта Московской Биржи.

In [5]:
import requests
import pandas as pd
from datetime import datetime

# Список тикеров
tickers = ["LKOH", "GMKN", "MTSS", "AFLT"]

# Параметры
iss_url = "https://iss.moex.com/iss/history/engines/stock/markets/shares/boards/tqbr/securities/"
tradingsession = 3

# Даты
start_date = "2024-05-18"
end_date = "2024-07-19"

# Список для хранения данных котировок
data = []

# Функция для получения котировок
def get_stock_quotes(ticker):
    url = f"{iss_url}{ticker}.json?from={start_date}&till={end_date}"
    response = requests.get(url)
    if response.status_code == 200:
        json_data = response.json()
        try:
            # Извлечение нужных столбцов
            quotes = json_data['history']['data']
            for quote in quotes:
                # Создаем словарь с нужными данными
                record = {
                    'TRADEDATE': quote[1],        # Дата торгов
                    'SECID': quote[3],             # Идентификатор ценной бумаги
                    'LEGALCLOSEPRICE': quote[9]  # Закрывающая цена
                }
                data.append(record)
        except KeyError as e:
            print(f"Ошибка при извлечении данных для {ticker}: {e}")
    else:
        print(f"Ошибка при получении данных для {ticker}: {response.status_code}")

# Получаем данные для каждого тикера
for ticker in tickers:
    get_stock_quotes(ticker)

In [6]:
# Преобразуем собранные данные в DataFrame
d1 = pd.DataFrame(data)
filtered_df = d1.pivot_table(index='TRADEDATE', columns='SECID', values='LEGALCLOSEPRICE', fill_value=0) #LEGALCLOSEPRICE - цена акций по итогам всех торгов
filtered_df = filtered_df.sort_index(ascending=False)
filtered_df = filtered_df[tickers] #ставим в том же порядке, как и задавали
filtered_df.head(5)

SECID,LKOH,GMKN,MTSS,AFLT
TRADEDATE,,,,
2024-07-19,6935.0,128.86,237.30,56.46
2024-07-18,6906.0,128.50,227.70,56.08
2024-07-17,6811.0,125.16,223.55,54.30
2024-07-16,6831.5,126.34,220.45,54.58
2024-07-15,6807.0,122.50,262.10,53.73
